# Coletando daos IBGE, Armazena no BigQuery, Preenchendo PPT e Disponibilizando em um chatBot do Telegran

**Este Artigo estará disponivel em tres etapas:**
- Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
- Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT
- Terceira Etapa: Disponibilização dos dados no ChatBoot do Telegran

### Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
Neste Artigo vamos coletar dados do IBGE utilizando o SIDRA (Sistema IBGE de Recuperação Automática) que é uma plataforma do IBGE criada com o objetivo de consultar as pesquisas criados e disponibilizadas pelo Instituto. A interface permite que sejam acessado os dados através de API's. 

O objetivo do post de hoje será mostrar como é possível acessar estes dados utilizando o Python, para exemplificar estarei utilizando a tabela de estimativa populacional por Municipio.

Existe um módulo python chamado sidrapy que permite baixar facilmente estes dados e realizar a importação direta para um dataframe Pandas.

Referências que utilizei para construir o Artigo:
- https://pypi.org/project/sidrapy/ => Documentação oficial módulo sidrapy no pipy
- https://sidrapy.readthedocs.io/pt_BR/latest/modules/table.html => documentação do sidrapy (achei esta mais detalhada)
- https://sidra.ibge.gov.br/acervo => Pagina para consultar tableas disponiveis
- https://apisidra.ibge.gov.br/ => Documentação da API

> Primeiramente precisaremos entrar no pompt de comando e instalar a bibliotecas necessarias através do arquivo requirements.txt atrvés dp comando 
> 
> ``` pip install -r requirements.txt ```
> 




**Importa Libs**

In [79]:
import sidrapy
from google.cloud import bigquery
from google.oauth2 import service_account

**Baixando a Tabela de Estimativa de Populção residente por Municipio**

Estaremos Utilizando os ultimos 6 anos da Tabela 6579 - População residente estimada agregada por Municipio

Descrição da Serie a ser utilizada: https://apisidra.ibge.gov.br/desctabapi.aspx?c=6579

Descrição dos parametros que utilizaremos:
- **table_code** = Código da tabela de onde se deseja extrair os dados, utilizaremos a *6579 - População Residente Estimada*
- **territorial_level** = Nível territorial do IBGE , utilizaremos a *6 - Município*
- **ibge_territorial_code** = Unidades territoriais do IBGE. É possível especificar múltiplos elementos utilizando vírgulas e utilizaremos toddos os disponiveis, estaremos utilizando o parametro "all" informando que queremos todos
- **period** = Períodos (meses, anos etc.) desejados. Caso não seja especificado, traz apenas os períodos mais recentes, utilizaremos os ultimos 6 anos
- **header** = Especifica se o resultado será precedido por um registro de cabeçalho. Caso não seja espefificado, traz o cabeçalho.
    - Opções:
        - ‘y’ - Traz o cabeçalho
        - ‘n’ - Não traz o cabeçalho'n'
    
    não iremos trazer o cabeçalho, pois vamos ajustar

In [80]:
dfPopulacaoResidente = sidrapy.get_table(table_code="6579", territorial_level="6", ibge_territorial_code="all", period="last 6", header='n')

**Ajustando data frame**

Quando baixamos o dataframe ele vem com os campos abaixo: 
- **NC**  = Nível Territorial (Código)
- **MN**  = Nível Territorial
- **MC**  = Unidade de Medida (Código)
- **MN**  = Unidade de Medida
- **V**   = Valor
- **D1C** = Unidade da Federação (Código)
- **D1N** = Unidade da Federação
- **D2C** = Ano (Código)
- **D2N** = Ano
- **D3C** = Variável (Código)	
- **D3N** = Variável	

Porém para nossa analise precisaremos apenas de alguns campos que são:
- **MN**  = Nível Territorial
- **V**   = Valor
- **D1N** = Unidade da Federação - Este campo sera dividido em Municipio e UF
- **D2N** = Ano
- **D3N** = Variável	

Então vamos tratar os dados e deixar apenas os campos necessarios e aproveitar para renomear as colunas


In [81]:
newColumns = dfPopulacaoResidente["D1N"].str.split(" - ", n = 1, expand = True)
dfPopulacaoResidente['municipio']= newColumns[0]
dfPopulacaoResidente['uf']= newColumns[1]
dfPopulacaoResidente.pop("D1N")
dfPopulacaoResidente.pop("NC")
dfPopulacaoResidente.pop("MC")
dfPopulacaoResidente.pop("D1C")
dfPopulacaoResidente.pop("D2C")
dfPopulacaoResidente.pop("D3C")
dfPopulacaoResidente.pop("NN")

dfPopulacaoResidente

,MN,V,D2N,D3N,municipio,uf
0,Pessoas,25506,2016,População residente estimada,Alta Floresta D'Oeste,RO
1,Pessoas,25437,2017,População residente estimada,Alta Floresta D'Oeste,RO
2,Pessoas,23167,2018,População residente estimada,Alta Floresta D'Oeste,RO
3,Pessoas,22945,2019,População residente estimada,Alta Floresta D'Oeste,RO
4,Pessoas,22728,2020,População residente estimada,Alta Floresta D'Oeste,RO
...,...,...,...,...,...,...
33415,Pessoas,3039444,2017,População residente estimada,Brasília,DF
33416,Pessoas,2974703,2018,População residente estimada,Brasília,DF
33417,Pessoas,3015268,2019,População residente estimada,Brasília,DF
33418,Pessoas,3055149,2020,População residente estimada,Brasília,DF


### Ajustando o nome das Colunas

In [83]:
dfPopulacaoResidente.columns = ['unidadeMedida',	'valor', 'ano', 'totalizador', 'municipio', 'uf']
dfPopulacaoResidente = dfPopulacaoResidente.astype({
    'valor': 'int64'
})

dfPopulacaoResidente.dtypes

unidadeMedida    object
valor             int64
ano              object
totalizador      object
municipio        object
uf               object
dtype: object

### Criando um Dataframe Novo com os dados Agrupados por UF

In [84]:
dfPopulacaoGroupUF = dfPopulacaoResidente.groupby(["unidadeMedida", 'ano', 'totalizador', 'uf']).agg({'valor': ['sum']}).reset_index()
dfPopulacaoGroupUF

,unidadeMedida,ano,totalizador,uf,valor
,,,,,sum
0,Pessoas,2016,População residente estimada,AC,816687
1,Pessoas,2016,População residente estimada,AL,3358963
2,Pessoas,2016,População residente estimada,AM,4001667
3,Pessoas,2016,População residente estimada,AP,782295
4,Pessoas,2016,População residente estimada,BA,15276566
...,...,...,...,...,...
157,Pessoas,2021,População residente estimada,RS,11466630
158,Pessoas,2021,População residente estimada,SC,7338473
159,Pessoas,2021,População residente estimada,SE,2338474


### Ajustando nome das colunas do DataFrame agrupado

In [90]:
dfPopulacaoGroupUF.columns = ['unidadeMedida', 'ano', 'totalizador', 'uf', 'valor']
dfPopulacaoGroupUF

,unidadeMedida,ano,totalizador,uf,valor
0,Pessoas,2016,População residente estimada,AC,816687
1,Pessoas,2016,População residente estimada,AL,3358963
2,Pessoas,2016,População residente estimada,AM,4001667
3,Pessoas,2016,População residente estimada,AP,782295
4,Pessoas,2016,População residente estimada,BA,15276566
...,...,...,...,...,...
157,Pessoas,2021,População residente estimada,RS,11466630
158,Pessoas,2021,População residente estimada,SC,7338473
159,Pessoas,2021,População residente estimada,SE,2338474
160,Pessoas,2021,População residente estimada,SP,46649132


### Autenticando no Google Cloud
**Neste ponto iremos utilizar a chave de serviço do GCP**

**Obs:** Caso não saiba gerar a chave pode utilizar o link a seguir:
- https://medium.com/@alexandretavares.bigdata/criando-uma-conta-de-servi%C3%A7o-no-gcp-eeac59f3cbcb


In [91]:
# Informe aqui o caminho onde a chave estará armazenada
# Recomendamos que guarde esta chave em um cofre digital para maior segurança
pathCredencial = "key/artmedium.json" 

credentials = service_account.Credentials.from_service_account_file(
    pathCredencial,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

### Gravando os Dados no BigQuery

In [ ]:
tbPessoasUF = 'raw.tb_nat_populacao_uf'
tbPessoasMunicipio = 'raw.tb_nat_populacao_municipio'

job_config = bigquery.LoadJobConfig(
    autodetect = True, 
    write_disposition = 'WRITE_TRUNCATE'
    )

job = client.load_table_from_dataframe(
    dfPopulacaoGroupUF, 
    tbPessoasUF, 
    job_config=job_config,
)

job2 = client.load_table_from_dataframe(
    dfPopulacaoResidente, 
    tbPessoasMunicipio, 
    job_config=job_config,
)